<a href="https://colab.research.google.com/github/pejmanrasti/EPU_ML_Angers_2023/blob/main/Jour_2/EPU_1_ImportDonneesBrutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quelles sont les étapes à suivre dans un projet de machine learning ?
1. **Quel est le projet ?**
  * Objectif ?
  * Comment le modèle sera-il utilisé en routine clinique ? (opérateur, contexte, contraintes, etc)
  * Le problème peut-il être résolu plus simplement ? (état des lieux des solutions actuelles)
  * Comment formuler le problème ? (apprentissage supervisé ou non, classification ou régression, type de données d'entrée, ...)
  * Comment mesurer la performance et quelle est la performance minimale recherchée ?

2. **Récupérer les données**
  * Evaluer quels types de données vont être nécessaires et en quelle quantité
  * Attention à la RGPD !
  * Convertir les données dans un format facilement manipulable
  * Automatiser la collecte autant que possible

  *A noter : en apprentissage supervisé, les données sont constituées de variables et de cibles. Les variables sont utilisées pour prédire les cibles.*

3. **Analyser les données**
  * Types de données présentes dans la base (quantitatives, qualitatives, entières ou réelles, etc)
  * Données manquantes
  * Données corrompues (données aberrantes, erreurs, etc) : nécessité d'avoir un oeil d'expert sur les données explorées
  * Visualiser les données pour évaluer les types de distributions (normale, beta, uniforme, etc)
  * Evaluer les corrélations entre les variables
  * Vérifier si le problème peut être résolu manuellement (ex : on découvre une forte corrélation entre une variable et les cibles)

4. **Préparer les données**
  * Nettoyer les données : supprimer les données aberrantes et remplacer ou supprimer les observations (lignes de la base) présentant des données manquantes.
  * Supprimer les données non pertinentes
  * Transformer les données : transformer des variables discrètes en variables continues ou inversement au besoin, appliquer des transformations potentiellement intéressantes (log, racine, carré, etc), combiner des variables entre-elles
  * Calibrer les variables
  * Séparer les données : conserver des données test qui ne seront jamais utilisées pour entrainer le modèle.

5. **Evaluer plusieurs modèles**
  * Sélectionner une liste d'algorithmes de différentes familles (linéaire, forêt aléatoire, SVM, réseau de neuronnes, etc)
  * Sélectionner les variables les plus pertinentes pour chaque algorithme
  * Dans un 1er temps, on peut entrainer les modèles en utilisant leur hyper-paramètres par défaut ou faire une recherche sur grille rapide.
  * Pour chaque modèle, comparer les performances obtenues sur les données d'entrainement via une validation croisée à k passes.
  * Sélectionner quelques uns des modèles testés les plus prometteurs

6. **Réglage fin des modèles**
  * Sélectionner les hyper-paramètres via une validation croisée sur les données d'entrainement. Commencer par évaluer les modèles sur une recherche grossière et affiner l'espace de recherche en fonction des résultats.
  * Eventuellement tester des méthodes ensemblistes qui permettent de combiner plusieurs modèles.
  * Evaluer les performances du modèle final sur les données test. Attention à ne pas utiliser les données test pendant le réglage du modèle ou à modifier le réglage du modèle en fonction des résultats obtenus sur les données test sinon la mesure de l'erreur de généralisation sur les données test sera biaisée de manière optimiste (sur-ajustement sur les données test).

7. **Surveiller son modèle**
  * Si on lance le modèle en routine clinique, il faut prendre garde à ce que les résultats ne dérivent pas. Cela peut arriver si les données d'entrée changent (nouveau détecteur, MAJ de l'algorithme de calcul, modification du type d'analyse réalisé, etc)
  * Si les données ont changé, il faut réentrainer le modèle




---



---



1. **Quel est le projet ?**

# Problématique
* **Prédire les CQ patients hors tolérances pour les plans VMAT des localisations pelviennes afin de réoptimiser ces plans directement**
* Données disponibles pour chaque plan actuellement dans la base de données patient : prescriptions, codes CIM, machine, énergie, positions du MLC, du bras et débit de dose pour tous les points de contrôles, distributions de dose, résultats du CQ pré-traitement, ...





---



---



2. **Récupérer les données**

In [ ]:
from pandas import read_csv
import re
import numpy

# Import de la base csv brute issue de l'extraction de données Aria
* Les données ont été collectées automatiquement via un executable C# et stockées dans un fichier csv

In [ ]:
# On importe le csv qui contient les données brutes via la fonction read_csv de la bibliothèque pandas
df = read_csv("/content/RAWDATABASE.csv",
              sep = ",",
              header=0
              )
df.head()

,Unnamed: 0,Patient_ID,Course_Name,Treatment_Plan_Name,Last_modification,CIM,CIM_description,Presc_name,Presc_cibles,Presc_DoseParFract,...,Gamma21glob10_MaxGamma,Gamma22glob10,Gamma22glob10_AverageGamma,Gamma22glob10_MaxGamma,Gamma33glob10,Gamma33glob10_AverageGamma,Gamma33glob10_MaxGamma,Gamma32glob10,Gamma32glob_AverageGamma,Gamma32glob10_MaxGamma
0,0,1007,cq/PDIP,NR,03/07/2023 17:43,NR,NR,NR,NR,NR,...,1.589,0.999,0.157,1.422,1.0,0.105,0.948,1.0,0.127,1.006
1,1,1007,cq/PDIP,NR,03/07/2023 17:43,NR,NR,NR,NR,NR,...,1.965,0.999,0.147,1.587,1.0,0.098,1.058,1.0,0.121,1.209
2,2,953,202107 MACHOIRE,RA30_mandDT,07/12/2021 16:06,C15.9,"Oesophage, sans precision ...",MACHOIRE D,Isocentre,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,953,202107 MACHOIRE,RA30_mandDT,07/12/2021 16:06,C15.9,"Oesophage, sans precision ...",MACHOIRE D,Isocentre,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,126,202211 COTES G,RA20 cotes G,11/21/2022 15:24:11,C79.5,Tumeur maligne secondaire des os et de la moel...,6e et 7e COTES G,Isocentre,4,...,1.860,0.999,0.149,1.410,1.0,0.099,0.940,1.0,0.128,1.137


In [ ]:
# La méthode info() permet d'avoir un résumé de la base de données : nbre de colonnes et de lignes, types de données présentes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2417 entries, 0 to 2416
Columns: 288 entries, Unnamed: 0 to  Gamma32glob10_MaxGamma
dtypes: float64(107), int64(10), object(171)
memory usage: 5.3+ MB


In [ ]:
# On jette un oeil sur les données qui sont des float
 [c for c in df.columns if df[c].dtype == "float64"]

['Plan_DoseParFraction',
 'Plan_CalcGrid',
 'MUperGy',
 'MUmax',
 'MUstd',
 'MUiqr',
 'AAV',
 'AAVmin',
 'AAVmax',
 'AAVstd',
 'AAViqr',
 'LSV',
 'LSVmin',
 'LSVmax',
 'LSVstd',
 'LSViqr',
 'MCS',
 'MCSmin',
 'MCSmax',
 'MCSstd',
 'MCSiqr',
 'AFW',
 'AFWmin',
 'AFWmax',
 'AFWstd',
 'AFWiqr',
 'ALT',
 'CAS',
 'CASmax',
 'CASstd',
 'CASiqr',
 'CLS',
 'CLSmin',
 'CLSmax',
 'CLSstd',
 'CLSiqr',
 'MAD',
 'MADmax',
 'MADstd',
 'MADiqr',
 'MFA',
 'MFAmax',
 'MFAstd',
 'MFAiqr',
 'SAS2',
 'SAS2max',
 'SAS2std',
 'SAS2iqr',
 'SAS5',
 'SAS5max',
 'SAS5std',
 'SAS5iqr',
 'SAS10',
 'SAS10max',
 'SAS10std',
 'SAS10iqr',
 'SAS20',
 'SAS20max',
 'SAS20std',
 'SAS20iqr',
 'EM',
 'EMmin',
 'EMmax',
 'EMstd',
 'EMiqr',
 'MIt02',
 'BI',
 'BImin',
 'BImax',
 'BIstd',
 'BIiqr',
 'BM',
 'BA',
 'BAmin',
 'BAmax',
 'BAstd',
 'BAiqr',
 'Gamma11loc10',
 ' Gamma11loc10_AverageGamma',
 ' Gamma11loc10_MaxGamma',
 'Gamma21loc10',
 ' Gamma21loc10_AverageGamma',
 ' Gamma21loc10_MaxGamma',
 'Gamma22loc10',
 ' Gamma22l

In [ ]:
# On jette un oeil pour voir quelles données sont des int
[c for c in df.columns if df[c].dtype == "int64"]

['Unnamed: 0',
 'Patient_ID',
 'MUmin',
 'CASmin',
 'MADmin',
 'MFAmin',
 'SAS2min',
 'SAS5min',
 'SAS10min',
 'SAS20min']

In [ ]:
# On jette un oeil pour voir quelles données sont classées comme "object" = non défini lors de l'import.
# En général les données qui comportent du texte sont importées comme des objets. Si on veut les utiliser, il faut dire à pandas que ce sont des string par exemple
# Ici toutes les PCM avec _prox ou _dist sont spécifiques à l'Halcyon. Pour le Novalis ces données sont étiquetées comme "NR" (paramétré lors de la collecte)
# Il est important de savoir comment les données ont été collectées pour anticiper des problèmes
[c for c in df.columns if df[c].dtype == "object"]

['Course_Name',
 'Treatment_Plan_Name',
 'Last_modification',
 'CIM',
 'CIM_description',
 'Presc_name',
 'Presc_cibles',
 'Presc_DoseParFract',
 'Presc_Fractions',
 'Plan_NbreFractions',
 'Plan_GroupeStruct',
 'Plan_ModeleCalc',
 'PDIP_Plan_Name',
 'PDIP_UID',
 'Technique',
 'Machine',
 'Beam',
 'AAV_prox',
 'AAVmin_prox',
 'AAVmax_prox',
 'AAVstd_prox',
 'AAViqr_prox',
 'AAV_dist',
 'AAVmin_dist',
 'AAVmax_dist',
 'AAVstd_dist',
 'AAViqr_dist',
 'LSV_prox',
 'LSVmin_prox',
 'LSVmax_prox',
 'LSVstd_prox',
 'LSViqr_prox',
 'LSV_dist',
 'LSVmin_dist',
 'LSVmax_dist',
 'LSVstd_dist',
 'LSViqr_dist',
 'MCS_prox',
 'MCSmin_prox',
 'MCSmax_prox',
 'MCSstd_prox',
 'MCSiqr_prox',
 'MCS_dist',
 'MCSmin_dist',
 'MCSmax_dist',
 'MCSstd_dist',
 'MCSiqr_dist',
 'AFW_prox',
 'AFWmin_prox',
 'AFWmax_prox',
 'AFWstd_prox',
 'AFWiqr_prox',
 'AFW_dist',
 'AFWmin_dist',
 'AFWmax_dist',
 'AFWstd_dist',
 'AFWiqr_dist',
 'ALT_prox',
 'ALT_dist',
 'CAS_prox',
 'CASmin_prox',
 'CASmax_prox',
 'CASstd_prox',


In [ ]:
# La méthode describe() permet de faire un résumé de la distribution des données de chaque colonne de la base
df.describe()

,Unnamed: 0,Patient_ID,Plan_DoseParFraction,Plan_CalcGrid,MUperGy,MUmin,MUmax,MUstd,MUiqr,AAV,...,Gamma21glob10_MaxGamma,Gamma22glob10,Gamma22glob10_AverageGamma,Gamma22glob10_MaxGamma,Gamma33glob10,Gamma33glob10_AverageGamma,Gamma33glob10_MaxGamma,Gamma32glob10,Gamma32glob_AverageGamma,Gamma32glob10_MaxGamma
count,2417.000000,2417.000000,2409.000000,2417.000000,2417.000000,2417.0,2417.000000,2417.000000,2417.000000,2417.000000,...,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000
mean,1208.000000,512.257344,2.842134,0.242056,258.237909,0.0,0.012373,0.001926,0.002118,0.363556,...,2.349695,0.995813,0.166091,1.902409,0.998428,0.110798,1.271814,0.997985,0.142485,1.446021
std,697.872123,296.827961,1.839080,0.033599,49.074723,0.0,0.006559,0.001260,0.001660,0.092088,...,1.045727,0.032604,0.155571,0.875619,0.029061,0.106005,0.624840,0.031708,0.138498,0.692240
min,0.000000,0.000000,1.800000,0.100000,132.155062,0.0,0.005982,0.000546,0.000009,0.059863,...,0.590000,0.146000,0.079000,0.435000,0.219000,0.053000,0.290000,0.165000,0.069000,0.361000
25%,604.000000,257.000000,2.000000,0.250000,223.681493,0.0,0.007902,0.001030,0.000965,0.302769,...,1.601000,0.996000,0.135750,1.326500,1.000000,0.090000,0.883750,0.999000,0.116000,0.993750
50%,1208.000000,510.000000,2.100000,0.250000,256.206928,0.0,0.010126,0.001595,0.001662,0.356074,...,2.161000,0.999000,0.157000,1.726000,1.000000,0.104500,1.150500,1.000000,0.134000,1.324500
75%,1812.000000,773.000000,3.000000,0.250000,288.122398,0.0,0.015112,0.002393,0.002722,0.415040,...,2.838000,1.000000,0.180000,2.262500,1.000000,0.120000,1.508250,1.000000,0.154000,1.738000
max,2416.000000,1018.000000,20.000000,0.250000,480.581511,0.0,0.090658,0.012491,0.016940,0.744318,...,10.000000,1.000000,4.607000,10.000000,1.000000,3.127000,10.000000,1.000000,4.085000,10.000000


# Sélection des données
* Toutes les données ne vont pas être utiles
* Certaines données sont manquantes ou dupliquées

## Données VMAT issues du Novalis et calculées en AAA.

In [ ]:
#On garde les données Novalis, en VMAT,l'algo AAA avec grille de calcul 0.25
#Comme on va filtrer en fonction des locs ensuite, on supprime également les lignes où le code CIM est absent
df = df[df.Technique!='DoseDynamic']
df = df[df.Machine=='NOVALIS']
df = df[df.Gamma22loc10!='NR']
df = df[df.Plan_CalcGrid==0.25]
df = df[df.Plan_ModeleCalc!='Acuros_1610']
df = df[df.CIM!='NR']

#On supprime toutes les colonnes qui ne sont pas des float (analyse préalable de la base = toutes les colonnes hors PCM, résultats gamma et ID patients)
for c in df.columns:
    if c !='CIM' : #On garde la colonne des codes CIM
        if df[c].dtype == 'object':
            df = df.drop(columns=[c])

In [ ]:
#Affichage des colonnes restantes dans df
list(df.columns)

['Unnamed: 0',
 'Patient_ID',
 'CIM',
 'Plan_DoseParFraction',
 'Plan_CalcGrid',
 'MUperGy',
 'MUmin',
 'MUmax',
 'MUstd',
 'MUiqr',
 'AAV',
 'AAVmin',
 'AAVmax',
 'AAVstd',
 'AAViqr',
 'LSV',
 'LSVmin',
 'LSVmax',
 'LSVstd',
 'LSViqr',
 'MCS',
 'MCSmin',
 'MCSmax',
 'MCSstd',
 'MCSiqr',
 'AFW',
 'AFWmin',
 'AFWmax',
 'AFWstd',
 'AFWiqr',
 'ALT',
 'CAS',
 'CASmin',
 'CASmax',
 'CASstd',
 'CASiqr',
 'CLS',
 'CLSmin',
 'CLSmax',
 'CLSstd',
 'CLSiqr',
 'MAD',
 'MADmin',
 'MADmax',
 'MADstd',
 'MADiqr',
 'MFA',
 'MFAmin',
 'MFAmax',
 'MFAstd',
 'MFAiqr',
 'SAS2',
 'SAS2min',
 'SAS2max',
 'SAS2std',
 'SAS2iqr',
 'SAS5',
 'SAS5min',
 'SAS5max',
 'SAS5std',
 'SAS5iqr',
 'SAS10',
 'SAS10min',
 'SAS10max',
 'SAS10std',
 'SAS10iqr',
 'SAS20',
 'SAS20min',
 'SAS20max',
 'SAS20std',
 'SAS20iqr',
 'EM',
 'EMmin',
 'EMmax',
 'EMstd',
 'EMiqr',
 'MIt02',
 'BI',
 'BImin',
 'BImax',
 'BIstd',
 'BIiqr',
 'BM',
 'BA',
 'BAmin',
 'BAmax',
 'BAstd',
 'BAiqr',
 'Gamma11loc10',
 ' Gamma11loc10_AverageGamma',

In [ ]:
#On garde seulement les PCMs, les CIMs et l'analyse gamma en 2%/2mm local
df = df.drop(columns=[
    'Patient_ID',
'Unnamed: 0',
 'Plan_DoseParFraction',
 'Plan_CalcGrid',
 'Gamma11loc10',
 ' Gamma11loc10_AverageGamma',
 ' Gamma11loc10_MaxGamma',
 'Gamma21loc10',
 ' Gamma21loc10_AverageGamma',
 ' Gamma21loc10_MaxGamma',
 #' Gamma22loc10',
 ' Gamma22loc10_AverageGamma',
 ' Gamma22loc10_MaxGamma',
 'Gamma33loc10',
 ' Gamma33loc10_AverageGamma',
 ' Gamma33loc10_MaxGamma',
 'Gamma32loc10',
 ' Gamma32loc10_AverageGamma',
 ' Gamma32loc10_MaxGamma',
 'Gamma11glob10',
 ' Gamma11glob10_AverageGamma',
 ' Gamma11glob10_MaxGamma',
 'Gamma21glob10',
 ' Gamma21glob10_AverageGamma',
 ' Gamma21glob10_MaxGamma',
 'Gamma22glob10',
 ' Gamma22glob10_AverageGamma',
 ' Gamma22glob10_MaxGamma',
 'Gamma33glob10',
 ' Gamma33glob10_AverageGamma',
 ' Gamma33glob10_MaxGamma',
 'Gamma32glob10',
 ' Gamma32glob_AverageGamma',
 ' Gamma32glob10_MaxGamma'])

df = df.reset_index()
df = df.drop(columns=['index'])

In [ ]:
df.head()

,CIM,MUperGy,MUmin,MUmax,MUstd,MUiqr,AAV,AAVmin,AAVmax,AAVstd,...,BImax,BIstd,BIiqr,BM,BA,BAmin,BAmax,BAstd,BAiqr,Gamma22loc10
0,C15.9,188.609536,0,0.015320,0.002248,0.002120,0.344026,0.164413,0.570164,0.111751,...,5.708773,1.057829,1.669608,0.546731,37.466117,17.5350,59.019999,11.509763,20.900000,NaN
1,C15.9,188.609536,0,0.014070,0.001965,0.001997,0.376546,0.209365,0.633108,0.085748,...,9.279901,1.523452,1.925466,0.525014,31.496345,19.2200,51.920000,6.456780,9.096875,NaN
2,C79.5,286.279357,0,0.025226,0.005673,0.007483,0.212154,0.004097,0.470404,0.144776,...,29.291912,7.922034,5.100115,0.680998,35.918076,0.7750,76.077499,24.333914,46.084374,0.995
3,C79.5,286.279357,0,0.031176,0.009321,0.016940,0.199329,0.004267,0.650373,0.205439,...,27.954796,11.082609,25.520850,0.734694,32.592787,0.7750,105.114999,33.919277,58.263750,0.990
4,C79.5,237.155273,0,0.015816,0.003171,0.004838,0.363556,0.055128,0.708376,0.197273,...,7.922318,1.369862,1.865898,0.572010,79.412440,12.9025,146.812498,40.413525,66.943749,0.999


## Localisation pelvienne

In [ ]:
# On ne garde que les lignes de données associées à la loc Pelvis (Prostate, Pelvis gyneco)

#Codes CIM associés à des locs pelviennes
code_pelv = ['C18.7', 'C19', 'C20', 'C21.0', 'C21.1', 'C51.8', 'C51.9', 'C52','C53.0','C53.1','C53.8','C53.9','C54.1',
            'C54.8','C54.9','C56','C61','C62.9','C66', 'C67.8','C67.9','C77.4','C77.5','C77.5','C79.1','C79.3','C79.5',
            'C82.9','D26.1']

#Conversion du code CIM de "Object" à "String"
df["CIM"] = df['CIM'].astype('string')



In [ ]:
# Quand on fait la collecte de données, notamment des données texte, on peut se retrouver avec des étrangetés
# Ici par exemple avec cet affichage on peut voit que des espaces sont présents avant et après les codes CIM (pas tous en réalité)
# Si on fait un filtre des codes CIM via la liste précédemment définie, aucune donnée ne sera retrouvée
print("XXX", df["CIM"][1], "XXX")

XXX C15.9            XXX


In [ ]:
#On supprime les espaces qu'il y a pour certains CIM
for i in range(len(df["CIM"])):
    df["CIM"][i] = re.sub(r"\s+$", "", df["CIM"][i])

#On filtre la base en fonction des codes CIM de la liste code_pelv
df = df[df.CIM.isin(code_pelv)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451 entries, 2 to 2253
Data columns (total 85 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CIM           1451 non-null   string 
 1   MUperGy       1451 non-null   float64
 2   MUmin         1451 non-null   int64  
 3   MUmax         1451 non-null   float64
 4   MUstd         1451 non-null   float64
 5   MUiqr         1451 non-null   float64
 6   AAV           1451 non-null   float64
 7   AAVmin        1451 non-null   float64
 8   AAVmax        1451 non-null   float64
 9   AAVstd        1451 non-null   float64
 10  AAViqr        1451 non-null   float64
 11  LSV           1451 non-null   float64
 12  LSVmin        1451 non-null   float64
 13  LSVmax        1451 non-null   float64
 14  LSVstd        1451 non-null   float64
 15  LSViqr        1451 non-null   float64
 16  MCS           1451 non-null   float64
 17  MCSmin        1451 non-null   float64
 18  MCSmax        1451 non-null 

In [ ]:
# Une fois que le filtrage en fonction des codes CIM a été fait, on n'a plus besoin de cette colonne donc on la supprime
df = df.drop(columns = ["CIM"])

## Suppression des lignes dupliquées et des lignes avec des valeurs manquantes
>Selon les cas, il est également possible de combler les éventuelles valeurs manquantes en leur attribuant une valeur (zéro, moyenne, médiane), des fonctions scikit-learn le font automatiquement.

In [ ]:
#Suppression des lignes dupliquées et des lignes avec des valeurs manquantes
df = df.drop_duplicates()
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 2 to 2253
Data columns (total 84 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MUperGy       1338 non-null   float64
 1   MUmin         1338 non-null   int64  
 2   MUmax         1338 non-null   float64
 3   MUstd         1338 non-null   float64
 4   MUiqr         1338 non-null   float64
 5   AAV           1338 non-null   float64
 6   AAVmin        1338 non-null   float64
 7   AAVmax        1338 non-null   float64
 8   AAVstd        1338 non-null   float64
 9   AAViqr        1338 non-null   float64
 10  LSV           1338 non-null   float64
 11  LSVmin        1338 non-null   float64
 12  LSVmax        1338 non-null   float64
 13  LSVstd        1338 non-null   float64
 14  LSViqr        1338 non-null   float64
 15  MCS           1338 non-null   float64
 16  MCSmin        1338 non-null   float64
 17  MCSmax        1338 non-null   float64
 18  MCSstd        1338 non-null 

# Sauvegarde de la base pelvis en csv

In [ ]:
df.to_csv('/content/DataSet_RegionPelvienne.csv')

* Un csv a été créé dans les fichiers du projet
* **Télécharger ce csv**